In [0]:
import dlt
from pyspark.sql.functions import col

In [0]:
import dlt

@dlt.table(
    name="machine_metadata",
    comment="Static metadata for production line machines and their monitored characteristics",
    table_properties={
        "quality": "silver"
    }
)
def machine_metadata():
    machines = [
        {
            "machine_id": 1,
            "machine_name": "Main Drive Motor",
            "machine_type": "Electric motor",
            "location": "Production Line A",
            "function": "Primary power source for the production line",
            "monitored_sensors": "temperature, vibration, power",
            "material_processed": "Steel / Aluminum alloys",
            "criticality": "High"
        },
        {
            "machine_id": 2,
            "machine_name": "Conveyor Belt Motor",
            "machine_type": "Electric motor",
            "location": "Material transport section",
            "function": "Moves raw and semi-finished parts between stations",
            "monitored_sensors": "temperature, vibration, speed",
            "material_processed": "Steel / Aluminum alloys",
            "criticality": "Medium"
        },
        {
            "machine_id": 3,
            "machine_name": "Hydraulic Press",
            "machine_type": "Hydraulic press system",
            "location": "Forming station",
            "function": "Applies high pressure for shaping metal components",
            "monitored_sensors": "temperature, pressure, cycle_count",
            "material_processed": "Steel / Aluminum alloys",
            "criticality": "High"
        },
        {
            "machine_id": 4,
            "machine_name": "CNC Spindle",
            "machine_type": "CNC machining spindle",
            "location": "Machining center",
            "function": "High-precision machining of metal parts",
            "monitored_sensors": "temperature, vibration, rpm",
            "material_processed": "Steel / Aluminum alloys",
            "criticality": "High"
        },
        {
            "machine_id": 5,
            "machine_name": "Cooling System Pump",
            "machine_type": "Industrial cooling pump",
            "location": "Cooling subsystem",
            "function": "Circulates coolant to maintain safe operating temperatures",
            "monitored_sensors": "temperature, flow_rate",
            "material_processed": "Coolant fluids",
            "criticality": "Medium"
        },
        {
            "machine_id": 6,
            "machine_name": "Packaging Unit Motor",
            "machine_type": "Electric motor",
            "location": "Packaging station",
            "function": "Drives packaging and final product handling",
            "monitored_sensors": "temperature, vibration, current",
            "material_processed": "Finished metal components",
            "criticality": "Medium"
        }
    ]

    return spark.createDataFrame(machines)
